In [56]:
#!pip install emoji
#!pip install pydantic==1.10.2.
#!pip3 install spacy
#!pip install gensim
#!pip install wordcloud
#!pip install openpyxl
!pip install XlsxWriter

In [57]:
#Removing emojis from text
#Refrence 1 : https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
#Refrence 2 : https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python

def give_emoji_free_text(text):
    """
    Removes emoji's from tweets
    Accepts:
        Text (tweets)
    Returns:
        Text (emoji free tweets)
    """
    emoji_list = [c for c in text if c in emoji.EMOJI_DATA]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text

def url_free_text(text):
    '''
    Cleans text from urls
    '''
    text = re.sub (r'^https?:\/\/.*[\r\n]*', '', text)
    
    #text = re.sub(r'http\S+', '', text)
    return text

In [58]:
from datetime import datetime
import numpy as np

def to_datetime(date):
    """
    Converts a numpy datetime64 object to a python datetime object 
    Input:
      date - a np.datetime64 object
    Output:
      DATE - a python datetime object
    """
    timestamp = ((date - np.datetime64('1970-01-01T00:00:00'))
                 / np.timedelta64(1, 's'))
    return datetime.utcfromtimestamp(timestamp)

In [59]:
from pandas import read_excel
data = read_excel("original_tweets.xlsx",sheet_name = 'original tweet data')

In [60]:
import pandas as pd
# Converting the dataset to pandas DataFrame and renaming the columns 
df_original_tweet = pd.DataFrame(data['text'].tolist())

df_original_tweet = df_original_tweet.rename(columns={0:'original_tweets'})
df_original_tweet = df_original_tweet.rename(columns={1:'tweet_id'})
df_original_tweet = df_original_tweet.rename(columns={2:'company_name'})#company_name
df_original_tweet = df_original_tweet.rename(columns={3:'created_at'})#created_at


df_original_tweet['company_name'] = data['company_name']
df_original_tweet['created_at'] = data['created_at']
df_original_tweet['tweet_id'] = data['tweet_id']
df_original_tweet.head(10)

,original_tweets,company_name,created_at,tweet_id
0,"Tomorrow, we walk out for real change. #google...",Google,2018-11-01T00:38:58.000Z,1057794087955427328
1,"We, Google employees and contractors, will wal...",Google,2018-11-01T01:19:09.000Z,1057804203895283712
2,Google employees and contractors will be leavi...,Google,2018-11-01T02:46:11.000Z,1057826104965246976
3,@sasha_feather @hypatiadotca Thank you so much...,Google,2018-11-01T03:29:24.000Z,1057836978954256385
4,Text (1/2):\n1. An end to Forced Arbitration i...,Google,2018-11-01T03:30:52.000Z,1057837349290287104
5,"(2/2):\n4. A clear, uniform, globally inclusiv...",Google,2018-11-01T03:31:55.000Z,1057837611891507200
6,"@hypatiadotca @sasha_feather That’s awesome, w...",Google,2018-11-01T03:36:14.000Z,1057838702330822656
7,More photos are coming in from the 40+ offices...,Google,2018-11-01T10:46:51.000Z,1057947069506355200
8,Another angle of the #GoogleWalkout at Google ...,Google,2018-11-01T10:52:29.000Z,1057948484731584514
9,"Google Dublin, we see you! Thanks for coming o...",Google,2018-11-01T11:32:33.000Z,1057958568551464961


In [61]:
import emoji
import re
# Apply the function above and get tweets free of emoji's
call_emoji_free = lambda x: give_emoji_free_text(x)

# Apply `call_emoji_free` which calls the function to remove all emoji's
df_original_tweet['emoji_free_tweets'] = df_original_tweet['original_tweets'].apply(call_emoji_free)

#Create a new column with url free tweets

df_original_tweet['sanitized_text'] = df_original_tweet['emoji_free_tweets'].apply(url_free_text)

In [62]:
# Filter by company and date
#company_name='Amazon'
#company_name = 'Amazon'
#df_original_tweet = df_original_tweet.loc[df_original_tweet['company_name'] == company_name]
df_original_tweet['created_at'] = pd.to_datetime(df_original_tweet['created_at']) # converting string in date object
df_original_tweet = df_original_tweet.sort_values(by="created_at")
df_original_tweet.head(5)

,original_tweets,company_name,created_at,tweet_id,emoji_free_tweets,sanitized_text
0,"Tomorrow, we walk out for real change. #google...",Google,2018-11-01 00:38:58+00:00,1057794087955427328,"Tomorrow, we walk out for real change. #google...","Tomorrow, we walk out for real change. #google..."
1,"We, Google employees and contractors, will wal...",Google,2018-11-01 01:19:09+00:00,1057804203895283712,"We, Google employees and contractors, will wal...","We, Google employees and contractors, will wal..."
2,Google employees and contractors will be leavi...,Google,2018-11-01 02:46:11+00:00,1057826104965246976,Google employees and contractors will be leavi...,Google employees and contractors will be leavi...
3,@sasha_feather @hypatiadotca Thank you so much...,Google,2018-11-01 03:29:24+00:00,1057836978954256385,@sasha_feather @hypatiadotca Thank you so much...,@sasha_feather @hypatiadotca Thank you so much...
4,Text (1/2):\n1. An end to Forced Arbitration i...,Google,2018-11-01 03:30:52+00:00,1057837349290287104,Text (1/2): 1. An end to Forced Arbitration in...,Text (1/2): 1. An end to Forced Arbitration in...


In [63]:
import spacy
import spacy.cli
spacy.cli.download("en_core_web_lg")
document = df_original_tweet['sanitized_text'].values.tolist()
#pprint.pprint(document)
# Load spacy
# Make sure to restart the runtime after running installations and libraries tab
#print(spacy.__version__)
nlp = spacy.load('en_core_web_lg')

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [64]:
"""
Import Gensim and Wordcloud to use their stopwords as well and use the combined stopwords of ALL as the variable:
ALL_STOP_WORDS
"""

import time
from spacy.tokenizer import Tokenizer
from gensim.parsing.preprocessing import STOPWORDS as SW
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)
# Timing Start
program_start_time = time.time()

# Tokenizer
tokenizer = Tokenizer(nlp.vocab)


# Custom stopwords
custom_stopwords = ['hi','\n','\n\n', '&amp;', ' ', '.', '-', 'got', "it's", 'it’s', "i'm", 'i’m', 'im', 'want', 'like', '$', '@','RT','rt','google','amazon','nyc','?','zurich','re','we']

# Customize stop words by adding to the default list
STOP_WORDS = nlp.Defaults.stop_words.union(custom_stopwords)

# ALL_STOP_WORDS = spacy + gensim + wordcloud
ALL_STOP_WORDS = STOP_WORDS.union(SW).union(stopwords)

In [65]:
df_original_tweet.head(10)

,original_tweets,company_name,created_at,tweet_id,emoji_free_tweets,sanitized_text
0,"Tomorrow, we walk out for real change. #google...",Google,2018-11-01 00:38:58+00:00,1057794087955427328,"Tomorrow, we walk out for real change. #google...","Tomorrow, we walk out for real change. #google..."
1,"We, Google employees and contractors, will wal...",Google,2018-11-01 01:19:09+00:00,1057804203895283712,"We, Google employees and contractors, will wal...","We, Google employees and contractors, will wal..."
2,Google employees and contractors will be leavi...,Google,2018-11-01 02:46:11+00:00,1057826104965246976,Google employees and contractors will be leavi...,Google employees and contractors will be leavi...
3,@sasha_feather @hypatiadotca Thank you so much...,Google,2018-11-01 03:29:24+00:00,1057836978954256385,@sasha_feather @hypatiadotca Thank you so much...,@sasha_feather @hypatiadotca Thank you so much...
4,Text (1/2):\n1. An end to Forced Arbitration i...,Google,2018-11-01 03:30:52+00:00,1057837349290287104,Text (1/2): 1. An end to Forced Arbitration in...,Text (1/2): 1. An end to Forced Arbitration in...
5,"(2/2):\n4. A clear, uniform, globally inclusiv...",Google,2018-11-01 03:31:55+00:00,1057837611891507200,"(2/2): 4. A clear, uniform, globally inclusive...","(2/2): 4. A clear, uniform, globally inclusive..."
6,"@hypatiadotca @sasha_feather That’s awesome, w...",Google,2018-11-01 03:36:14+00:00,1057838702330822656,"@hypatiadotca @sasha_feather That’s awesome, w...","@hypatiadotca @sasha_feather That’s awesome, w..."
7,More photos are coming in from the 40+ offices...,Google,2018-11-01 10:46:51+00:00,1057947069506355200,More photos are coming in from the 40+ offices...,More photos are coming in from the 40+ offices...
8,Another angle of the #GoogleWalkout at Google ...,Google,2018-11-01 10:52:29+00:00,1057948484731584514,Another angle of the #GoogleWalkout at Google ...,Another angle of the #GoogleWalkout at Google ...
9,"Google Dublin, we see you! Thanks for coming o...",Google,2018-11-01 11:32:33+00:00,1057958568551464961,"Google Dublin, we see you! Thanks for coming o...","Google Dublin, we see you! Thanks for coming o..."


In [66]:
tokens = []

for doc in tokenizer.pipe(df_original_tweet['sanitized_text'], batch_size=500):
    doc_tokens = []    
    for token in doc: 
        if token.text.lower() not in STOP_WORDS:
            doc_tokens.append(token.text.lower()) 
    tokens.append(doc_tokens)

# Makes tokens column
df_original_tweet['tokens'] = tokens

# Make tokens a string again
df_original_tweet['tokens_back_to_text'] = [' '.join(map(str, l)) for l in df_original_tweet['tokens']]

# Timing End
program_end_time = time.time()

# View df
#df[['sanitized_text','tokens','tokens_back_to_text']]
df_original_tweet.shape

(3172, 8)

In [67]:
#Build bigrams and trigrams
import gensim
bigram_phrases = gensim.models.Phrases(df_original_tweet['tokens'], min_count=5, threshold=100) # higher threshold fewer phrases.
#trigram_phrases = gensim.models.Phrases(bigram_phrases[df['tokens']], threshold=100)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
#trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return [bigram[doc] for doc in texts]

#def make_trigrams(texts):
#    return [trigram[bigram[doc]] for doc in texts]

data_words_unigrams_bigrams = make_bigrams(df_original_tweet['tokens'])
df_original_tweet['data_words_uni_bi_tri_grams'] = data_words_unigrams_bigrams

In [68]:
#Filter out unigrams
def filter_unigrams(df):
    new_tokens_list = []
    tokens = df['data_words_uni_bi_tri_grams']
    for row in tokens:
        new_tokens = []
        for word in row:
            if word.find('_')!=-1:
                new_tokens.append(word)
        if new_tokens:        
            new_tokens_list.append(new_tokens)
        else:
            df.drop(row, axis=5)
    return new_tokens_list

def filter_unigrams2(df):
    new_tokens_list = []
    data = df
    for index, row in data.iterrows():
        new_tokens = []
        for word in row['data_words_uni_bi_tri_grams']:
            if word.find('_')!=-1:
                new_tokens.append(word)
        if new_tokens:        
            new_tokens_list.append(new_tokens)
        else:
            #print(index)
            df.drop(index,inplace = True)
    
    return new_tokens_list


df_original_tweet['data_words_bigrams'] = filter_unigrams2(df_original_tweet)
                
#pprint.pprint(data_words_bigrams)

#print(df.shape[0])
#df

In [69]:
#print(data_words_bigrams)

#These are names of places or people.
bigrams_to_remove_set = {"amznclimate_fridaysforfuture",
                         "copley_square","googlewac_msworkers",
                         "emahlee_marencosta",
                     "jeff_bezos","meredith_clairewaves","timnit_gebru","mountain_view",
                         "san_bernardino",
                     "dr_timnit","dr_gebru","dr_gebrus",
                         "san_francisco","new_york",
                         "bay_area","timnitgebru",
                         "meredith","wayfairwalkout", "tweetsofsandra_week","riot_games", "githubdropice_notechforice","notechforice_githubdropice" }
                        # {"near_warehouses","stay_tuned",
                        # "board_directors","temps_vendors","software_engineer",
                        # "press_release","shareholder_meeting","gas_companies","tweetsofsandra_week"}

            
def filter_places_and_people_names(df):
    data = df_original_tweet
    data_words_bigrams_filtered = []
    for index, row in data.iterrows():
        tmp = []
        for word in row['data_words_bigrams']:
            if word not in bigrams_to_remove_set:
                tmp.append(word)
        if tmp:
            data_words_bigrams_filtered.append(tmp)
        else:
            df_original_tweet.drop(index, inplace = True)
    
    return data_words_bigrams_filtered
            
        
df_original_tweet['data_words_bigrams_filtered'] = filter_places_and_people_names(df_original_tweet)


In [70]:

text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)

In [71]:
df_original_tweet.head(20)

,original_tweets,company_name,created_at,tweet_id,emoji_free_tweets,sanitized_text,tokens,tokens_back_to_text,data_words_uni_bi_tri_grams,data_words_bigrams,data_words_bigrams_filtered
3,@sasha_feather @hypatiadotca Thank you so much...,Google,2018-11-01 03:29:24+00:00,1057836978954256385,@sasha_feather @hypatiadotca Thank you so much...,@sasha_feather @hypatiadotca Thank you so much...,"[@sasha_feather, @hypatiadotca, thank, reminde...",@sasha_feather @hypatiadotca thank reminder! w...,"[@sasha_feather, @hypatiadotca, thank, reminde...",[@sasha_feather],[@sasha_feather]
4,Text (1/2):\n1. An end to Forced Arbitration i...,Google,2018-11-01 03:30:52+00:00,1057837349290287104,Text (1/2): 1. An end to Forced Arbitration in...,Text (1/2): 1. An end to Forced Arbitration in...,"[text, (1/2):, 1., end, forced, arbitration, c...",text (1/2): 1. end forced arbitration cases ha...,"[text, (1/2):, 1., end, forced_arbitration, ca...","[forced_arbitration, sexual_harassment]","[forced_arbitration, sexual_harassment]"
6,"@hypatiadotca @sasha_feather That’s awesome, w...",Google,2018-11-01 03:36:14+00:00,1057838702330822656,"@hypatiadotca @sasha_feather That’s awesome, w...","@hypatiadotca @sasha_feather That’s awesome, w...","[@hypatiadotca, @sasha_feather, that’s, awesom...","@hypatiadotca @sasha_feather that’s awesome, w...","[@hypatiadotca, @sasha_feather, that’s, awesom...",[@sasha_feather],[@sasha_feather]
20,Google employees at the company's U.K. headqua...,Google,2018-11-01 14:04:17+00:00,1057996752735870976,Google employees at the company's U.K. headqua...,Google employees at the company's U.K. headqua...,"[employees, company's, u.k., headquarters, sta...",employees company's u.k. headquarters staging ...,"[employees, company's, u.k., headquarters, sta...",[sexual_harassment],[sexual_harassment]
33,PHOTOS: Worldwide #GoogleWalkout over sexual h...,Google,2018-11-01 15:22:57+00:00,1058016551796989952,PHOTOS: Worldwide #GoogleWalkout over sexual h...,PHOTOS: Worldwide #GoogleWalkout over sexual h...,"[photos:, worldwide, #googlewalkout, sexual, h...",photos: worldwide #googlewalkout sexual harass...,"[photos:, worldwide, #googlewalkout, sexual_ha...","[sexual_harassment,]","[sexual_harassment,]"
54,Women's rights are human rights #GoogleWalkout...,Google,2018-11-01 15:47:40+00:00,1058022770192326656,Women's rights are human rights #GoogleWalkout...,Women's rights are human rights #GoogleWalkout...,"[women's, rights, human, rights, #googlewalkou...",women's rights human rights #googlewalkout nyc...,"[women's, rights, human_rights, #googlewalkout...",[human_rights],[human_rights]
74,Google employees walked out of the tech compan...,Google,2018-11-01 16:51:51+00:00,1058038924793298945,Google employees walked out of the tech compan...,Google employees walked out of the tech compan...,"[employees, walked, tech, company's, atlanta, ...",employees walked tech company's atlanta office...,"[employees, walked, tech, company's, atlanta, ...",[sexual_harassment],[sexual_harassment]
81,I and others feel buoyed by their powerful sup...,Google,2018-11-01 17:25:11+00:00,1058047313241874432,I and others feel buoyed by their powerful sup...,I and others feel buoyed by their powerful sup...,"[feel, buoyed, powerful, support,, today, glad...","feel buoyed powerful support, today glad chose...","[feel, buoyed, powerful, support,, today, glad...","[thank_you,]","[thank_you,]"
155,#GoogleWalkout at #YouTube to end sexual haras...,Google,2018-11-01 18:53:50+00:00,1058069623978033152,#GoogleWalkout at #YouTube to end sexual haras...,#GoogleWalkout at #YouTube to end sexual haras...,"[#googlewalkout, #youtube, end, sexual, harass...",#googlewalkout #youtube end sexual harassment ...,"[#googlewalkout, #youtube, end, sexual_harassm...",[sexual_harassment],[sexual_harassment]
162,"Thank you, Google DC! #GoogleWalkout https://t...",Google,2018-11-01 19:03:06+00:00,1058071953444167681,"Thank you, Google DC! #GoogleWalkout https://t...","Thank you, Google DC! #Goog

In [72]:
df_original_tweet['grams_to_text'] = [' '.join(map(str, l)) for l in df_original_tweet['data_words_uni_bi_tri_grams']]

In [73]:
import re
sanatized_text = []
for text in df_original_tweet['grams_to_text']:
    result = re.sub(r"http\S+", "", text)
    sanatized_text.append(result)

df_original_tweet['sanitized_text_without_url'] =  sanatized_text   

In [74]:
df_original_tweet.head(100)

,original_tweets,company_name,created_at,tweet_id,emoji_free_tweets,sanitized_text,tokens,tokens_back_to_text,data_words_uni_bi_tri_grams,data_words_bigrams,data_words_bigrams_filtered,grams_to_text,sanitized_text_without_url
3,@sasha_feather @hypatiadotca Thank you so much...,Google,2018-11-01 03:29:24+00:00,1057836978954256385,@sasha_feather @hypatiadotca Thank you so much...,@sasha_feather @hypatiadotca Thank you so much...,"[@sasha_feather, @hypatiadotca, thank, reminde...",@sasha_feather @hypatiadotca thank reminder! w...,"[@sasha_feather, @hypatiadotca, thank, reminde...",[@sasha_feather],[@sasha_feather],@sasha_feather @hypatiadotca thank reminder! w...,@sasha_feather @hypatiadotca thank reminder! w...
4,Text (1/2):\n1. An end to Forced Arbitration i...,Google,2018-11-01 03:30:52+00:00,1057837349290287104,Text (1/2): 1. An end to Forced Arbitration in...,Text (1/2): 1. An end to Forced Arbitration in...,"[text, (1/2):, 1., end, forced, arbitration, c...",text (1/2): 1. end forced arbitration cases ha...,"[text, (1/2):, 1., end, forced_arbitration, ca...","[forced_arbitration, sexual_harassment]","[forced_arbitration, sexual_harassment]",text (1/2): 1. end forced_arbitration cases ha...,text (1/2): 1. end forced_arbitration cases ha...
6,"@hypatiadotca @sasha_feather That’s awesome, w...",Google,2018-11-01 03:36:14+00:00,1057838702330822656,"@hypatiadotca @sasha_feather That’s awesome, w...","@hypatiadotca @sasha_feather That’s awesome, w...","[@hypatiadotca, @sasha_feather, that’s, awesom...","@hypatiadotca @sasha_feather that’s awesome, w...","[@hypatiadotca, @sasha_feather, that’s, awesom...",[@sasha_feather],[@sasha_feather],"@hypatiadotca @sasha_feather that’s awesome, w...","@hypatiadotca @sasha_feather that’s awesome, w..."
20,Google employees at the company's U.K. headqua...,Google,2018-11-01 14:04:17+00:00,1057996752735870976,Google employees at the company's U.K. headqua...,Google employees at the company's U.K. headqua...,"[employees, company's, u.k., headquarters, sta...",employees company's u.k. headquarters staging ...,"[employees, company's, u.k., headquarters, sta...",[sexual_harassment],[sexual_harassment],employees company's u.k. headquarters staging ...,employees company's u.k. headquarters staging ...
33,PHOTOS: Worldwide #GoogleWalkout over sexual h...,Google,2018-11-01 15:22:57+00:00,1058016551796989952,PHOTOS: Worldwide #GoogleWalkout over sexual h...,PHOTOS: Worldwide #GoogleWalkout over sexual h...,"[photos:, worldwide, #googlewalkout, sexual, h...",photos: worldwide #googlewalkout sexual harass...,"[photos:, worldwide, #googlewalkout, sexual_ha...","[sexual_harassment,]","[sexual_harassment,]",photos: worldwide #googlewalkout sexual_harass...,photos: worldwide #googlewalkout sexual_harass...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
547,We stand with the brave @GoogleWalkout organiz...,Amazon,2019-04-23 04:41:38+00:00,1120548259964039170,We stand with the brave @GoogleWalkout organiz...,We stand with the brave @GoogleWalkout organiz...,"[stand, brave, @googlewalkout, organizers, @me...",stand brave @googlewalkout organizers @mer__ed...,"[stand, brave, @googlewalkout, organizers, @me...",[@mer__edith],[@mer__edith],stand brave @googlewalkout organizers @mer__ed...,stand brave @googlewalkout organizers @mer__ed...
550,This is dangerous for all women in tech. Googl...,Microsoft,2019-04-23 16:30:23+00:00,1120726621860667394,This is dangerous for all women in tech. Googl...,This is dangerous for all women in tech. Googl...,"[dangerous, women, tech., leading, company, re...",dangerous women tech. leading company retaliat...,"[dangerous, women, tech., leading, company, re...",[sexual_harassment],[sexual_harassment],dangerous women tech. leading company retaliat...,dangerous women tech. leading company retaliat...
551,We stand with the brave @GoogleWalkout organiz...,Microsoft,2019-04-23 17:14:48+00:00,1120737798439354368,We stand with the brave @GoogleWalkout organiz...,We stand with the 

In [77]:
df_santanize = df_original_tweet[['tweet_id','original_tweets','company_name','sanitized_text_without_url','grams_to_text']].copy()

In [78]:

df_santanize['tweet_id'] = df_santanize['tweet_id'].apply(str)

# create excel writer object
writer1 = pd.ExcelWriter('sanatized_data.xlsx')
# write dataframe to excel
df_santanize.to_excel(writer1, sheet_name = 'sanatized_text', engine="xlsxwriter")

# save the excel
writer1.save()
#writer.close()
print('DataFrame is written successfully to Excel File.')

DataFrame is written successfully to Excel File.
